## All the links for reprice

Looker:
https://lookerstudio.google.com/u/0/reporting/078c8c24-8210-462a-a814-e31ee1066a34/page/p_e4t9i225ad

Admin: https://admin.hourloop.com/

Settlement: https://seller.walmart.com/wfsLite/reports


Order Report: https://seller.walmart.com/order-management/details?orderGroups=All

Inventory:
https://seller.walmart.com/wfsLite/manage-inventory

Inbound receipts:
https://seller.walmart.com/wfsLite/reports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import packages
!pip install pytz
import pytz
import math
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import gc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Looker & Admin


In [3]:
# read looker
df_looker= pd.read_csv('/content/drive/MyDrive/Reprice/Walmart_Reprice NEW_Table.csv')

# drop vars
df_looker.drop('avg_cost', axis=1, inplace=True)

In [4]:
# --- Vendor ---
# export vender nan to excel sheet2
df_nan = df_looker[df_looker['Vendor'].isna()]

# remove those obs with vendor nan
df_looker = df_looker.dropna(subset=['Vendor'])

In [5]:
# Avg cost (admin)
# select vars
df_admin= pd.read_csv('/content/drive/MyDrive/Reprice/all product costs.csv')[['ASIN','Avg Cost']]

# merge
df_looker = df_looker.merge(df_admin, on='ASIN', how='left')

# if admin nan, take vendor price
df_looker['Avg Cost'].fillna(df_looker['vendor_price'], inplace=True)

<ipython-input-5-137ff8fe25dd>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_looker['Avg Cost'].fillna(df_looker['vendor_price'], inplace=True)


In [6]:
# 清掉df_admin變數，釋放RAM
del df_admin
gc.collect()

0

In [7]:
# remove the duplicates SKU
df_looker.sort_values(by=['SKU', 'Avg Cost'], ascending=[True, False], inplace=True)
df_looker.drop_duplicates(subset='SKU', keep='first', inplace=True)

In [8]:
# Lowest_amazon_price
# replace nan with 0
df_looker['lowest_amazon_price'].fillna(0, inplace=True)

<ipython-input-8-482a80437c3e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_looker['lowest_amazon_price'].fillna(0, inplace=True)


In [9]:
# MAP, Is_map_enforced
df_looker['map'].isna()
df_looker.loc[(df_looker['map'].isna()), 'map'] = df_looker['resolved_map']


# CA+ DB
# CA
indices= df_looker[df_looker['Vendor']== 'Cathedral Art'].index

for index in indices:
    df_looker.at[index, 'map'] = df_looker.at[index, 'vendor_price'] * 2 + 1.99
    df_looker.at[index, 'is_map_enforced'] = True

# DB
df_db= pd.read_excel('/content/drive/MyDrive/Reprice/MAP.xlsx')[['SKU','MAP']]
# merge df_looker
df_looker = df_looker.merge(df_db, on='SKU', how='left')
# replace
df_list = df_db['SKU'].tolist()
df_looker['map'] = df_looker.apply(lambda row: row['MAP'] if row['SKU'] in df_list else row['map'], axis=1)
# drop map
df_looker.drop('MAP', axis=1, inplace=True)

In [10]:
# 清掉df_db變數，釋放RAM
del df_db
gc.collect()

190

## 2. Shipping Fee

In [11]:
# new settlement15
df15= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202503.csv', skiprows=3)
df15['Partner GTIN']= df15['Partner GTIN'].replace(['"', '='], '', regex=True)
df15['Partner GTIN'] = pd.to_numeric(df15['Partner GTIN'], errors='coerce', downcast='integer')
df15['Walmart.com PO #']= df15['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df15= df15.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement14
df14= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202502.csv', skiprows=3)
df14['Partner GTIN']= df14['Partner GTIN'].replace(['"', '='], '', regex=True)
df14['Partner GTIN'] = pd.to_numeric(df14['Partner GTIN'], errors='coerce', downcast='integer')
df14['Walmart.com PO #']= df14['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df14= df14.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement13
df13= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202501.csv', skiprows=3)
df13['Partner GTIN']= df13['Partner GTIN'].replace(['"', '='], '', regex=True)
df13['Partner GTIN'] = pd.to_numeric(df13['Partner GTIN'], errors='coerce', downcast='integer')
df13['Walmart.com PO #']= df13['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df13= df13.rename(columns={'Walmart.com PO #': 'PO Number'})


# new settlement12
df12= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202412.csv', skiprows=3)
df12['Partner GTIN']= df12['Partner GTIN'].replace(['"', '='], '', regex=True)
df12['Partner GTIN'] = pd.to_numeric(df12['Partner GTIN'], errors='coerce', downcast='integer')
df12['Walmart.com PO #']= df12['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df12= df12.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement11
df11= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202411.csv', skiprows=3)
df11['Partner GTIN']= df11['Partner GTIN'].replace(['"', '='], '', regex=True)
df11['Partner GTIN'] = pd.to_numeric(df11['Partner GTIN'], errors='coerce', downcast='integer')
df11['Walmart.com PO #']= df11['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df11= df11.rename(columns={'Walmart.com PO #': 'PO Number'})


# new settlement10
df10= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202410.csv', skiprows=3)
df10['Partner GTIN']= df10['Partner GTIN'].replace(['"', '='], '', regex=True)
df10['Partner GTIN'] = pd.to_numeric(df10['Partner GTIN'], errors='coerce', downcast='integer')
df10['Walmart.com PO #']= df10['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df10= df10.rename(columns={'Walmart.com PO #': 'PO Number'})


# new settlement9
df9= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202409.csv', skiprows=3)
df9['Partner GTIN']= df9['Partner GTIN'].replace(['"', '='], '', regex=True)
df9['Partner GTIN'] = pd.to_numeric(df9['Partner GTIN'], errors='coerce', downcast='integer')
df9['Walmart.com PO #']= df9['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df9= df9.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement8
df8= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202408.csv', skiprows=3)
df8['Partner GTIN']= df8['Partner GTIN'].replace(['"', '='], '', regex=True)
df8['Partner GTIN'] = pd.to_numeric(df8['Partner GTIN'], errors='coerce', downcast='integer')
df8['Walmart.com PO #']= df8['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df8= df8.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement7
df7= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202407.csv', skiprows=3)
df7['Partner GTIN']= df7['Partner GTIN'].replace(['"', '='], '', regex=True)
df7['Partner GTIN'] = pd.to_numeric(df7['Partner GTIN'], errors='coerce', downcast='integer')
df7['Walmart.com PO #']= df7['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df7= df7.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement6
df6= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202406.csv', skiprows=3)
df6['Partner GTIN']= df6['Partner GTIN'].replace(['"', '='], '', regex=True)
df6['Partner GTIN'] = pd.to_numeric(df6['Partner GTIN'], errors='coerce', downcast='integer')
df6['Walmart.com PO #']= df6['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df6= df6.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement5
df5= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202405.csv', skiprows=3)
df5['Partner GTIN']= df5['Partner GTIN'].replace(['"', '='], '', regex=True)
df5['Partner GTIN'] = pd.to_numeric(df5['Partner GTIN'], errors='coerce', downcast='integer')
df5['Walmart.com PO #']= df5['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df5= df5.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement4
df4= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202404.csv', skiprows=3)
df4['Partner GTIN']= df4['Partner GTIN'].replace(['"', '='], '', regex=True)
df4['Partner GTIN'] = pd.to_numeric(df4['Partner GTIN'], errors='coerce', downcast='integer')
df4['Walmart.com PO #']= df4['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df4= df4.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement3
df3= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0312.csv', skiprows=3)
df3['Partner GTIN']= df3['Partner GTIN'].replace(['"', '='], '', regex=True)
df3['Partner GTIN'] = pd.to_numeric(df3['Partner GTIN'], errors='coerce', downcast='integer')
df3['Walmart.com PO #']= df3['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df3= df3.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement2
df2= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0312.csv', skiprows=3)
df2['Partner GTIN']= df2['Partner GTIN'].replace(['"', '='], '', regex=True)
df2['Partner GTIN'] = pd.to_numeric(df2['Partner GTIN'], errors='coerce', downcast='integer')
df2['Walmart.com PO #']= df2['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df2= df2.rename(columns={'Walmart.com PO #': 'PO Number'})

# new settlement
df0= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0227.csv', skiprows=3)
df0['Partner GTIN']= df0['Partner GTIN'].replace(['"', '='], '', regex=True)
df0['Partner GTIN'] = pd.to_numeric(df0['Partner GTIN'], errors='coerce', downcast='integer')
df0['Walmart.com PO #']= df0['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df0= df0.rename(columns={'Walmart.com PO #': 'PO Number'})

# shipping fee
df1= pd.read_csv('/content/drive/MyDrive/Reprice/settlement.csv', skiprows=3)
df1['Partner GTIN']= df1['Partner GTIN'].replace(['"', '='], '', regex=True)
df1['Partner GTIN'] = pd.to_numeric(df1['Partner GTIN'], errors='coerce', downcast='integer')
df1['Walmart.com PO #']= df1['Walmart.com PO #'].replace(['"', '='], '', regex=True)
df1= df1.rename(columns={'Walmart.com PO #': 'PO Number'})

# concat
df1= pd.concat([df0, df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], ignore_index=True)


# shipping fee%
df_pl_inbound= df1[df1['Transaction Type']== 'InboundTransportationFee']
df_pl_inbound= df_pl_inbound.groupby('PO Number')['Net Payable'].sum().reset_index()

In [12]:
# inbound receipts
df_inbound= pd.read_csv('/content/drive/MyDrive/Reprice/inboundReceipts.csv')
df_inbound['GTIN']= df_inbound['GTIN'].replace(['"', '='], '', regex=True)
df_inbound['GTIN'] = pd.to_numeric(df_inbound['GTIN'], errors='coerce', downcast='integer')

# merge
df_inbound= df_inbound.merge(df_looker[['GTIN','Avg Cost','Vendor']], on='GTIN', how='left')
df_inbound= df_inbound[~df_inbound['Vendor'].isna()]
df_inbound['Inventory Value']= df_inbound['Received Units']*df_inbound['Avg Cost']
df_inbound.head(2)


,Inbound Order ID,PO Number,Original PO Number,GTIN,UPC,SKU,Description,PO Create Date,PO Delivered Date,PO Status,Expected Units,Checked in units at ICC,Received Units,Damaged Units,Avg Cost,Vendor,Inventory Value
0,HOU-1692865274361,2544015GDM,NaN,9203000015,"=""009203000015""",4T-1EYC-4RD6,"McGinley Mills 1.3"" W Acetate Satin Ribbon, Re...",2023-08-24,2023-09-29,RECEIVED,22,0,22,0,14.39,burton + BURTON,316.58
1,HOU-1692865274361,2544015GDM,NaN,9203000022,"=""009203000022""",7M-8EH0-CF85,"McGinley Mills 2.5"" W Acetate Satin Ribbon, Wh...",2023-08-24,2023-09-29,RECEIVED,21,0,21,0,12.76,burton + BURTON,267.96


In [13]:
comfy_list = [
    'A&A Story',
    'Beetle',
    'Gong Maw',
    'Greenland',
    'Hanlong',
    'Homenice',
    'Hung Plus',
    'Lian Xing',
    'Newwish',
    'Sunset Vista (Comfy Hour)',
    'Teammann',
    'Tingle',
    'Vikson Blinds',
    'WUJI',
    'Yi Xiang'
]
df_comfy = pd.DataFrame({'Vendor': comfy_list})
df_inbound.loc[df_inbound['Vendor'].isin(comfy_list), 'Vendor'] = 'Comfy Hour'

In [14]:
# return max vendor
df_maxvendor= df_inbound.groupby('PO Number')['Vendor'].apply(lambda x: x.value_counts().idxmax()).reset_index()

# inven value
df_valuevendor= df_inbound.groupby('PO Number')['Inventory Value'].sum().reset_index()

# merge vendor, inven value
df_inbound= df_maxvendor.merge(df_valuevendor, how='left',on='PO Number')
df_inbound.head(3)

,PO Number,Vendor,Inventory Value
0,2121353GDM,Clover,973.45
1,2127011GDM,Intelex,3128.88
2,2310775GDM,Clover,0.00


In [15]:
del df_valuevendor
gc.collect()

31

In [16]:
# merge settlement, inbound
df_pl_inbound= df_pl_inbound.merge(df_inbound, how='left',on='PO Number')

# group by shipping no and return sum
df_pl_inbound= df_pl_inbound.groupby('Vendor').agg({'Net Payable': 'sum','Inventory Value': 'sum'}).reset_index()

# shipping fee&%
df_pl_inbound['Shipping%']= df_pl_inbound['Net Payable']/df_pl_inbound['Inventory Value']

In [17]:
 # add comfy brands
 df_pl_inbound = pd.concat([df_pl_inbound, df_comfy], ignore_index=True)

 # fillin with comfy hour %
 df_pl_inbound.loc[df_pl_inbound['Vendor'].isin(comfy_list), 'Shipping%'] = df_pl_inbound.loc[df_pl_inbound['Vendor'] == 'Comfy Hour', 'Shipping%'].values[0]
 df_ship= df_pl_inbound[['Vendor','Shipping%']]

# 10/29 fix_檢查 'Comfy Hour' 是否存在於 df_pl_inbound['Vendor']
if not df_pl_inbound.loc[df_pl_inbound['Vendor'] == 'Comfy Hour', 'Shipping%'].empty:
    comfy_hour_shipping = df_pl_inbound.loc[df_pl_inbound['Vendor'] == 'Comfy Hour', 'Shipping%'].values[0]
    df_pl_inbound.loc[df_pl_inbound['Vendor'].isin(comfy_list), 'Shipping%'] = comfy_hour_shipping
else:
    # 如果找不到 'Comfy Hour'，可以選擇填入預設值，例如 0 或 np.nan
    df_pl_inbound.loc[df_pl_inbound['Vendor'].isin(comfy_list), 'Shipping%'] = 0
    df_ship= df_pl_inbound[['Vendor','Shipping%']]

In [18]:
# make vendors lowercase to match pa tracking
df_ship['Vendor'] = df_ship['Vendor'].str.lower()
df_looker['Vendor'] = df_looker['Vendor'].str.lower()

# merge
df = df_looker.merge(df_ship, on='Vendor', how='left')

# if charge nan fill 0
df['Shipping%'] = df['Shipping%'].fillna(0.05)

<ipython-input-18-e1fd3755af16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ship['Vendor'] = df_ship['Vendor'].str.lower()


## 3. Settlement & Inventory

In [19]:
# new settlement15
df15= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202503.csv', skiprows=3)
df15['Partner GTIN']= df15['Partner GTIN'].replace(['"', '='], '', regex=True)
df15['Partner GTIN'] = pd.to_numeric(df15['Partner GTIN'], errors='coerce', downcast='integer')


# new settlement14
df14= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202502.csv', skiprows=3)
df14['Partner GTIN']= df14['Partner GTIN'].replace(['"', '='], '', regex=True)
df14['Partner GTIN'] = pd.to_numeric(df14['Partner GTIN'], errors='coerce', downcast='integer')


# new settlement13
df13= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202501.csv', skiprows=3)
df13['Partner GTIN']= df13['Partner GTIN'].replace(['"', '='], '', regex=True)
df13['Partner GTIN'] = pd.to_numeric(df13['Partner GTIN'], errors='coerce', downcast='integer')


# new settlement12
df12= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202412.csv', skiprows=3)
df12['Partner GTIN']= df12['Partner GTIN'].replace(['"', '='], '', regex=True)
df12['Partner GTIN'] = pd.to_numeric(df12['Partner GTIN'], errors='coerce', downcast='integer')


# new settlement11
df11= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202411.csv', skiprows=3)
df11['Partner GTIN']= df11['Partner GTIN'].replace(['"', '='], '', regex=True)
df11['Partner GTIN'] = pd.to_numeric(df11['Partner GTIN'], errors='coerce', downcast='integer')


# new settlement10
df10= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202410.csv', skiprows=3)
df10['Partner GTIN']= df10['Partner GTIN'].replace(['"', '='], '', regex=True)
df10['Partner GTIN'] = pd.to_numeric(df10['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement9
df9= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202409.csv', skiprows=3)
df9['Partner GTIN']= df9['Partner GTIN'].replace(['"', '='], '', regex=True)
df9['Partner GTIN'] = pd.to_numeric(df9['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement8
df8= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202408.csv', skiprows=3)
df8['Partner GTIN']= df8['Partner GTIN'].replace(['"', '='], '', regex=True)
df8['Partner GTIN'] = pd.to_numeric(df8['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement7
df7= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202407.csv', skiprows=3)
df7['Partner GTIN']= df7['Partner GTIN'].replace(['"', '='], '', regex=True)
df7['Partner GTIN'] = pd.to_numeric(df7['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement6
df6= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202406.csv', skiprows=3)
df6['Partner GTIN']= df6['Partner GTIN'].replace(['"', '='], '', regex=True)
df6['Partner GTIN'] = pd.to_numeric(df6['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement5
df5= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202405.csv', skiprows=3)
df5['Partner GTIN']= df5['Partner GTIN'].replace(['"', '='], '', regex=True)
df5['Partner GTIN'] = pd.to_numeric(df5['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement4
df4= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_202404.csv', skiprows=3)
df4['Partner GTIN']= df4['Partner GTIN'].replace(['"', '='], '', regex=True)
df4['Partner GTIN'] = pd.to_numeric(df4['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement3
df3= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0326.csv', skiprows=3)
df3['Partner GTIN']= df3['Partner GTIN'].replace(['"', '='], '', regex=True)
df3['Partner GTIN'] = pd.to_numeric(df3['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement2
df2= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0312.csv', skiprows=3)
df2['Partner GTIN']= df2['Partner GTIN'].replace(['"', '='], '', regex=True)
df2['Partner GTIN'] = pd.to_numeric(df2['Partner GTIN'], errors='coerce', downcast='integer')

# new settlement
df0= pd.read_csv('/content/drive/MyDrive/Reprice/settlement_0227.csv', skiprows=3)
df0['Partner GTIN']= df0['Partner GTIN'].replace(['"', '='], '', regex=True)
df0['Partner GTIN'] = pd.to_numeric(df0['Partner GTIN'], errors='coerce', downcast='integer')

# old settlement
df1= pd.read_csv('/content/drive/MyDrive/Reprice/settlement.csv', skiprows=3)
df1['Partner GTIN']= df1['Partner GTIN'].replace(['"', '='], '', regex=True)
df1['Partner GTIN'] = pd.to_numeric(df1['Partner GTIN'], errors='coerce', downcast='integer')

# concat to old settlement
df1= pd.concat([df0, df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], ignore_index=True)

In [20]:
# del df0,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13
# gc.collect()

In [21]:
# fulfillment + prep fee
df_settle = df1[(df1['Transaction Type'] == 'FulfillmentFee') | (df1['Transaction Type'] == 'PrepServiceFee')]
# select vars
df_settle2= df_settle[['Partner GTIN','Transaction Type','Reason Code','Detail','Qty','Net Payable']]
# select vars
df_settle= df_settle[['Partner GTIN','Transaction Type','Reason Code','Qty','Net Payable']]
# rename
df_settle2= df_settle2.rename(columns={'Partner GTIN': 'GTIN'})
# rename
df_settle= df_settle.rename(columns={'Partner GTIN': 'GTIN'})

In [22]:
# wfs
df_wfs= df_settle2[df_settle2['Transaction Type']== 'FulfillmentFee']
# sum obs with same gtin
df_wfs['Net Sum'] = df_wfs.groupby(['GTIN'])['Net Payable'].transform('sum')
# see num of duplicates
df_wfs['Duplicates Count'] = df_wfs.groupby(['GTIN'])['Net Payable'].transform('count')
# take avg
# df_wfs['Avg. Net Payment']= round(df_wfs['Net Sum']/ df_wfs['Duplicates Count'], 2)
df_wfs['Avg. Net Payment']=  df_wfs['Net Payable']/ df_wfs['Qty']
# When 'Detail'= 'Under$10', WFS_Fee -$1
df_wfs.loc[df_wfs['Detail'] == 'Under$10', 'Avg. Net Payment'] = df_wfs.loc[df_wfs['Detail'] == 'Under$10', 'Avg. Net Payment'] - 1
# rename
df_wfs= df_wfs.rename(columns={'Avg. Net Payment': 'WFS Fee Settlement'})
df_wfs.drop(['Detail'], axis=1, inplace=True)
df_wfs.head(10)

<ipython-input-22-a04ea95c61ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wfs['Net Sum'] = df_wfs.groupby(['GTIN'])['Net Payable'].transform('sum')
<ipython-input-22-a04ea95c61ab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wfs['Duplicates Count'] = df_wfs.groupby(['GTIN'])['Net Payable'].transform('count')
<ipython-input-22-a04ea95c61ab>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,GTIN,Transaction Type,Reason Code,Qty,Net Payable,Net Sum,Duplicates Count,WFS Fee Settlement
0,4.554433e+10,FulfillmentFee,NaN,1,3.45,175.80,36,3.45
1,4.554476e+10,FulfillmentFee,NaN,1,3.45,58.65,17,3.45
2,4.554433e+10,FulfillmentFee,NaN,1,3.45,175.80,36,3.45
3,4.554489e+10,FulfillmentFee,NaN,1,3.45,228.00,64,3.45
4,2.249493e+10,FulfillmentFee,NaN,1,7.35,132.30,16,7.35
5,2.249493e+10,FulfillmentFee,NaN,1,7.35,132.30,16,7.35
6,2.249493e+10,FulfillmentFee,NaN,1,7.35,132.30,16,7.35
7,4.554485e+10,FulfillmentFee,NaN,1,3.45,55.20,16,3.45
8,2.890100e+10,FulfillmentFee,NaN,1,3.45,155.35,43,3.45
9,4.554489e+10,FulfillmentFee,NaN,1,3.45,228.00,64,3.45


In [23]:
# drop wfs duplicates
df_wfs.drop_duplicates(subset=['GTIN'], keep='first', inplace=True)

In [24]:
# prep
df_prep= df_settle[df_settle['Transaction Type']== 'PrepServiceFee']
# check if duplicates max is 2
df_prep['Duplicates Count'] = df_prep.groupby(['GTIN'])['Net Payable'].transform('count')
# prep by 1 unit
df_prep['Avg. Net Payment']= df_prep['Net Payable']/ df_prep['Qty']

<ipython-input-24-2de1544a3ac2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep['Duplicates Count'] = df_prep.groupby(['GTIN'])['Net Payable'].transform('count')
<ipython-input-24-2de1544a3ac2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep['Avg. Net Payment']= df_prep['Net Payable']/ df_prep['Qty']


In [25]:
# drop unwanted vars
df_prep.drop(['Qty', 'Net Payable', 'Duplicates Count'], axis=1, inplace=True)

# sum 2 types of prep fee
df_prep['Prep Fee'] = df_prep.groupby('GTIN')['Avg. Net Payment'].transform('sum')

# drop prep duplicates
df_prep.drop_duplicates(subset=['GTIN'], keep='first', inplace=True)

# select vars
df_wfs= df_wfs[['GTIN','WFS Fee Settlement']]
df_prep= df_prep[['GTIN','Prep Fee']]

<ipython-input-25-5241d502cd87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.drop(['Qty', 'Net Payable', 'Duplicates Count'], axis=1, inplace=True)
<ipython-input-25-5241d502cd87>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep['Prep Fee'] = df_prep.groupby('GTIN')['Avg. Net Payment'].transform('sum')
<ipython-input-25-5241d502cd87>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [26]:
# merge wfs df
df = df.merge(df_wfs, on='GTIN', how='left')

# replace wfs fee with net payable if have one, remain wfs if dont
for index, row in df.iterrows():
    if not pd.isna(row['WFS Fee Settlement']):
        df.at[index, 'WFS Fee'] = row['WFS Fee Settlement']

# merge prep
df = df.merge(df_prep, on='GTIN', how='left')

# fill nan with 0
df['Prep Fee'] = df['Prep Fee'].fillna(0)

In [27]:
# real shipping fee
df['Shipping Fee']= df['Avg Cost']*df['Shipping%']

# on hand or inbound
df['on hand or inbound'] = df.apply(lambda row: True if row['Available to sell'] > 0 or row['Inbound units'] > 0 else False, axis=1)

##Item Report

In [28]:
df1= pd.read_csv('/content/drive/MyDrive/Restock/ItemReport.csv')

# select vars
df_item= df1[['Reviews Count', 'Average Rating', 'Publish Status','Buy Box Item Price','SKU']]

<ipython-input-28-f43fdc691bef>:1: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df1= pd.read_csv('/content/drive/MyDrive/Restock/ItemReport.csv')


In [29]:
df = df.merge(df_item, on='SKU', how='left')

# fill nan with 0
df['Reviews Count'] = df['Reviews Count'].fillna(0)
df['Average Rating'] = df['Average Rating'].fillna(0)

In [30]:
# del df_item
# gc.collect()

In [31]:
df

,Vendor,Item ID,SKU,GTIN,ASIN,is_map_enforced,lowest_amazon_price,vendor_price,map,Available to sell,...,Avg Cost,Shipping%,WFS Fee Settlement,Prep Fee,Shipping Fee,on hand or inbound,Reviews Count,Average Rating,Publish Status,Buy Box Item Price
0,companion group,194621677,00-03LJ-ITNV,50016751182,B00FJDER84,False,0.00,6.50,16.25,0,...,5.85,0.050000,NaN,0.0,0.292500,False,0.0,0.0,PUBLISHED,13.00
1,old mountain,880350577,00-08TL-750D,43502101783,B004O6LZLO,False,0.00,70.00,0.00,0,...,70.00,0.097206,NaN,0.0,6.804422,True,0.0,0.0,PUBLISHED,140.00
2,up with paper,1831634378,00-0TU1-36LC,48641558252,B07F6XWXZG,False,0.00,5.50,0.00,0,...,4.25,0.050000,NaN,0.0,0.212500,False,0.0,0.0,PUBLISHED,16.00
3,roman,365791985,00-0XYS-IH7A,89945502794,B001D00VKA,True,18.62,9.25,0.00,0,...,7.59,0.050000,NaN,0.0,0.379500,False,0.0,0.0,PUBLISHED,18.45
4,manual,40008541,00-3E3K-IIV2,725734449344,B00763NY3E,False,17.89,6.00,0.00,0,...,5.10,0.072528,NaN,0.0,0.369893,False,0.0,0.0,PUBLISHED,17.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183398,design design,1828530074,ZZZB-LLEI-IISH,732296420384,B09T2PS2ZF,False,0.00,2.98,0.00,0,...,2.36,0.050000,NaN,0.0,0.118000,False,0.0,0.0,PUBLISHED,10.96
183399,badia,31009081,ZZZO-5HQH-MO,33844000622,B00G7TRM7M,False,0.00,0.48,0.00,0,...,0.48,0.050000,NaN,0.0,0.024000,False,0.0,0.0,UNPUBLISHED,0.88
183400,christian art gifts,1703121165,ZZZP-6CMJ-KP,9781639522200,1639522204,False,45.99,42.50,84.99,0,...,34.00,0.050000,NaN,0.0,1.700000,False,1.0,5.0,PUBLISHED,123.78
183401,ka-bar,588079038,ZZZU-8WTX-TN1-0DM,617717299278,B08L43G3DV,False,21.99,18.51,37.02,0,...,17.60,0.050000,NaN,0.0,0.880000,False,0.0,0.0,PUBLISHED,26.11


##Looker_Current

In [32]:
df_current= pd.read_csv('/content/drive/MyDrive/Reprice/Walmart_Reprice NEW_Current.csv')
# select vars

df_current= df_current[['identity_sku','is_discontinue', 'GTIN', 'current_price','min_order_qty','last_30_days_instock_days',	'last_30_days_sales',	'last_30_days_units_sales',	'last_7_days_instock_days',	'last_7_days_sales',	'last_7_days_units_sales']]

In [33]:
# remove the duplicates SKU
# df_current.sort_values(by=['identity_sku', 'Avg Cost'], ascending=[True, False], inplace=True)
df_current.drop_duplicates(subset='identity_sku', keep='first', inplace=True)

In [34]:
df = df.merge(df_current, left_on='SKU', right_on='identity_sku', how='left')

# fill nan with 0
df['last_30_days_instock_days'] = df['last_30_days_instock_days'].fillna(0)
df['last_30_days_sales'] = df['last_30_days_sales'].fillna(0)
df['last_30_days_units_sales'] = df['last_30_days_units_sales'].fillna(0)
df['last_7_days_instock_days'] = df['last_7_days_instock_days'].fillna(0)
df['last_7_days_sales'] = df['last_7_days_sales'].fillna(0)
df['last_7_days_units_sales'] = df['last_7_days_units_sales'].fillna(0)
df['is_discontinue'] = df['is_discontinue'].fillna('True')

In [35]:

# # 定義每次讀取和處理的批次大小
# chunk_size = 100

# # Create an empty DataFrame to store the results if it doesn't exist
# if 'df' not in locals():
#     df = pd.DataFrame()

#  使用 chunksize 分批讀取 df_current 並合併
# for i, chunk in enumerate(pd.read_csv('/content/drive/MyDrive/Reprice/Walmart_Reprice NEW_Current.csv',
#                          usecols=['is_discontinue', 'GTIN', 'current_price', 'min_order_qty',
#                                   'last_30_days_instock_days', 'last_30_days_sales', 'last_30_days_units_sales',
#                                   'last_7_days_instock_days', 'last_7_days_sales', 'last_7_days_units_sales'],
#                          chunksize=chunk_size)):
#     # If it's the first chunk, assign it directly to df
#     if i == 0:
#         df = chunk
#     # For subsequent chunks, merge with existing df, handling duplicate columns
#     else:
#         df = df.merge(chunk, on='GTIN', how='left', suffixes=('', '_y'))  # Add suffixes only to new columns
#         # Drop duplicate columns from the right DataFrame after merging
#         df = df[[c for c in df.columns if not c.endswith('_y')]]

#     # 刪除目前批次並進行垃圾回收
#     del chunk
#     gc.collect()

In [36]:
df

,Vendor,Item ID,SKU,GTIN_x,ASIN,is_map_enforced,lowest_amazon_price,vendor_price,map,Available to sell,...,is_discontinue,GTIN_y,current_price,min_order_qty,last_30_days_instock_days,last_30_days_sales,last_30_days_units_sales,last_7_days_instock_days,last_7_days_sales,last_7_days_units_sales
0,companion group,194621677,00-03LJ-ITNV,50016751182,B00FJDER84,False,0.00,6.50,16.25,0,...,True,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,old mountain,880350577,00-08TL-750D,43502101783,B004O6LZLO,False,0.00,70.00,0.00,0,...,True,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,up with paper,1831634378,00-0TU1-36LC,48641558252,B07F6XWXZG,False,0.00,5.50,0.00,0,...,False,4.864156e+10,16.00,6.0,0.0,0.0,0.0,0.0,0.0,0.0
3,roman,365791985,00-0XYS-IH7A,89945502794,B001D00VKA,True,18.62,9.25,0.00,0,...,False,8.994550e+10,23.50,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,manual,40008541,00-3E3K-IIV2,725734449344,B00763NY3E,False,17.89,6.00,0.00,0,...,False,7.257344e+11,19.94,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183398,design design,1828530074,ZZZB-LLEI-IISH,732296420384,B09T2PS2ZF,False,0.00,2.98,0.00,0,...,True,7.322964e+11,10.96,6.0,0.0,0.0,0.0,0.0,0.0,0.0
183399,badia,31009081,ZZZO-5HQH-MO,33844000622,B00G7TRM7M,False,0.00,0.48,0.00,0,...,True,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
183400,christian art gifts,1703121165,ZZZP-6CMJ-KP,9781639522200,1639522204,False,45.99,42.50,84.99,0,...,True,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
183401,ka-bar,588079038,ZZZU-8WTX-TN1-0DM,617717299278,B08L43G3DV,False,21.99,18.51,37.02,0,...,True,6.177173e+11,37.02,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Order Report

In [37]:
df1= pd.read_excel('/content/drive/MyDrive/Reprice/PO_Data.xlsx')

In [38]:
# select vars
df_order= df1[['SKU','Order Date','Item Cost']]
# convert type
df_order['Order Date'] = pd.to_datetime(df_order['Order Date'])

<ipython-input-38-cdfb2943131a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order['Order Date'] = pd.to_datetime(df_order['Order Date'])


In [39]:
# only retain most recent order, remove duplicates
df_order.sort_values(by=['SKU', 'Order Date'], ascending=[True, False], inplace=True)
df_order.drop_duplicates(subset='SKU', keep='first', inplace=True)

<ipython-input-39-8770e3534d32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order.sort_values(by=['SKU', 'Order Date'], ascending=[True, False], inplace=True)
<ipython-input-39-8770e3534d32>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order.drop_duplicates(subset='SKU', keep='first', inplace=True)


In [40]:
today = datetime.now()

# us time zone
us_time_zone = pytz.timezone('America/New_York')

# convert current date
today = today.replace(tzinfo=pytz.utc).astimezone(us_time_zone)

In [41]:
# no sale in 14
today = datetime.today()
df_order['Sold in last'] = (today - df_order['Order Date']).dt.days

<ipython-input-41-67180cb6807c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order['Sold in last'] = (today - df_order['Order Date']).dt.days


In [42]:
# merge
df = df.merge(df_order, on='SKU', how='left')
df['Order Date']= pd.to_datetime(df['Order Date']).dt.to_period('D')
gc.collect()

# add cost
df['Cost']= round(df['Avg Cost']+ df['WFS Fee'], 2)
df['Cost'] = np.where(df['current_price'] < 10, df['Cost'] + 1, df['Cost'])

#add current price margin
df['Current Price Margin']= round(0.85 - df['Cost']/df['current_price'],2)

#add AMZ_lowest margin
df['AMZ lowest Margin']= round(0.85 - df['Cost']/df['lowest_amazon_price'],2)

#add BB Price Margin
df['Buy Box Item Price Margin']= round(0.85 - df['Cost']/df['Buy Box Item Price'],2)

#add Breakeven Price
df['Breakeven']= round(df['Cost']/0.85,2)

#add Final Map
df['is_map_enforced'] = df['is_map_enforced'].replace({True: 'TRUE', False: 'FALSE'})
df['Final Map'] = np.where(df['is_map_enforced'] == 'TRUE', df['map'], '0')

In [43]:
df.head(3)

,Vendor,Item ID,SKU,GTIN_x,ASIN,is_map_enforced,lowest_amazon_price,vendor_price,map,Available to sell,...,last_7_days_units_sales,Order Date,Item Cost,Sold in last,Cost,Current Price Margin,AMZ lowest Margin,Buy Box Item Price Margin,Breakeven,Final Map
0,companion group,194621677,00-03LJ-ITNV,50016751182,B00FJDER84,FALSE,0.0,6.5,16.25,0,...,0.0,NaT,NaN,NaN,10.80,NaN,-inf,0.02,12.71,0
1,old mountain,880350577,00-08TL-750D,43502101783,B004O6LZLO,FALSE,0.0,70.0,0.00,0,...,0.0,NaT,NaN,NaN,86.15,NaN,-inf,0.23,101.35,0
2,up with paper,1831634378,00-0TU1-36LC,48641558252,B07F6XWXZG,FALSE,0.0,5.5,0.00,0,...,0.0,NaT,NaN,NaN,7.70,0.37,-inf,0.37,9.06,0


In [44]:
# organised  df

# reorganised cols 10/11(+ review ranking + item status)

#df= df[['Vendor','Item ID','SKU','ASIN','vendor_price', 'Avg Cost','Cost','Shipping%','Shipping Fee','Prep Fee','Reviews Count','Average Rating','Publish Status','is_discontinue',
#        'map', 'is_map_enforced', 'Available to sell','Inbound units','Days of supply', 'WFS Fee','on hand or inbound',
#        'Order Date','Sold in last','lowest_amazon_price','AMZ lowest Margin','current_price','Current Price Margin']]

df= df[['Vendor','Item ID','SKU','ASIN','vendor_price', 'Avg Cost','WFS Fee','Cost','Reviews Count','Average Rating','Publish Status','is_discontinue','min_order_qty',
        'map', 'is_map_enforced','Final Map', 'Available to sell','Inbound units','Days of supply','on hand or inbound',
        'Order Date','Item Cost','Sold in last','Buy Box Item Price','lowest_amazon_price','AMZ lowest Margin','current_price','Current Price Margin',
        'last_30_days_instock_days',	'last_30_days_sales',	'last_30_days_units_sales',	'last_7_days_instock_days',	'last_7_days_sales',	'last_7_days_units_sales']]


# rename
columns_to_rename = {
    'vendor_price': 'Vendor Price',
    'Avg Cost': 'Avg. Cost',
    'Shipping%': 'Shipping Fee %',
    'lowest_amazon_price': 'Lowest Amazon Price',
    'map': 'MAP',
    'is_map_enforced': 'MAP Enforced',
    'Available to sell': 'Available to Sell',
    'min_order_qty': 'MOQ',
    'Item Cost': 'Last Sold Price',
}

for old_col, new_col in columns_to_rename.items():
    df = df.rename(columns={old_col: new_col})

In [45]:
df.head(3)

,Vendor,Item ID,SKU,ASIN,Vendor Price,Avg. Cost,WFS Fee,Cost,Reviews Count,Average Rating,...,Lowest Amazon Price,AMZ lowest Margin,current_price,Current Price Margin,last_30_days_instock_days,last_30_days_sales,last_30_days_units_sales,last_7_days_instock_days,last_7_days_sales,last_7_days_units_sales
0,companion group,194621677,00-03LJ-ITNV,B00FJDER84,6.5,5.85,4.95,10.80,0.0,0.0,...,0.0,-inf,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,old mountain,880350577,00-08TL-750D,B004O6LZLO,70.0,70.00,16.15,86.15,0.0,0.0,...,0.0,-inf,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,up with paper,1831634378,00-0TU1-36LC,B07F6XWXZG,5.5,4.25,3.45,7.70,0.0,0.0,...,0.0,-inf,16.0,0.37,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# export to my drive
df.to_excel('/content/drive/MyDrive/Results/Costs.xlsx', index=False)

# export sheet1.
# drop obs with no inven
df.to_excel('Re-price Table.xlsx', index=False, sheet_name='Main')
# df.to_excel('/content/drive/MyDrive/Reprice/Re-price Table.xlsx', engine='openpyxl', index=False)
df = df.loc[df['Available to Sell'] != 0]

In [47]:
df['Cost_fee'] = df['Cost'] / 0.85
condition = df['Cost_fee'] < 10
df.loc[condition, 'Cost_fee'] = df.loc[condition, 'Cost_fee'] + 1.2

<ipython-input-47-63ad14a66d8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cost_fee'] = df['Cost'] / 0.85


## Export

In [48]:
# # remove unwanted SKUs, manually paste here
# # --- incentive ---
# # export to sheet3
# df_list= pd.read_excel('/content/drive/MyDrive/Reprice/Special_case.xlsx',sheet_name='Incentives')['SKU'].tolist()
# df_incentive = df[df['SKU'].isin(df_list)] # new df
# df= df[~df['SKU'].isin(df_list)] # drop incentive

# # --- fixed price --- for MAP item
# # export to sheet 4.
# df_list= pd.read_excel('/content/drive/MyDrive/Reprice/Special_case.xlsx',sheet_name='Fixed price')['SKU'].tolist()

# df_fix = df[df['SKU'].isin(df_list)] # new df
# df= df[~df['SKU'].isin(df_list)] # drop fixed price

# # --- regular bump-up ---
# # export to sheet 5.
# df_list= pd.read_excel('/content/drive/MyDrive/Reprice/Special_case.xlsx',sheet_name='regular bump up')['SKU'].tolist()
# df_regbump = df[df['SKU'].isin(df_list)] # new df
# df = df[~df['SKU'].isin(df_list)]

# # --- KA Bump---
# # KA TODAY
# df_ka= df[df['Vendor']== 'kurt adler']
# df_list= df_ka[df_ka['Sold in last']== 0]['SKU'].tolist()
# df_kabump = df_ka[df_ka['SKU'].isin(df_list)]

In [49]:
# # read highest/ latest price
# df1= pd.read_excel('/content/drive/MyDrive/Reprice/HighestPrice.xlsx')
# df1 = df1.drop_duplicates(keep='last')
# df_kabump= df_kabump.merge(df1, on='SKU', how='left')
# gc.collect()

In [50]:
# ----- KA Bump -----
#df_kabump['Max']= df_kabump['Price']*1.3

In [51]:
# # --- Season ---
# # export to sheet 6.
# df_list= pd.read_excel('/content/drive/MyDrive/Reprice/SeasonalItems.xlsx', sheet_name='Sheet2')['SKU'].tolist()
# df_season = df[df['SKU'].isin(df_list)]

# df_storage= pd.read_excel('/content/drive/MyDrive/Reprice/StorageUntilNov.xlsx')[['SKU','Storage Until Nov']]
# df_season= df_season.merge(df_storage, on='SKU', how='left')
# gc.collect()
# df_season['Storage Until Nov'].fillna(round(df_season['Storage Until Nov'].mean(),3), inplace=True) #當時的Min price = Storage Fee+Cost
# df_season['Storage+Cost']= round(df_season['Storage Until Nov']+df_season['Cost_fee'],2) #暫時用不到，先不重寫

In [52]:
# # --- Non Season ---
# df= df[~df['SKU'].isin(df_list)]

# # inbound days
# df['Days of supply']= df['Days of supply'].astype(str).replace('Over', '', regex=True)
# df['Days of supply']= df['Days of supply'].astype(str).replace('Not enough data',46, regex=True)
# df['Days of supply']= df['Days of supply'].astype(int)

# # >45 days
# df_45 = df.loc[
#     ((df['Days of supply'] >= 45) & (df['Sold in last'] >= 45)) |
#     ((df['Days of supply'] >= 45) & (pd.isnull(df['Sold in last'])))
#     ]

# df_list= df_45['SKU'].tolist()
# df = df[~df['SKU'].isin(df_list)]

In [53]:
# # functions

# # fun1
# def return_max(df,str1, ins1, ins2, out1):
#     df[str1] = np.where(
#         map_condition, df['MAP'] + 0.01,
#         np.where(
#             alp0_condition, np.maximum(df[ins2],df[out1]),
#             np.maximum(
#                 np.minimum(df[ins1], df[ins2]),
#                 df[out1])))
# # fun1.2
# def return_max(df,str1, ins1, ins2, out1):
#     df[str1] = np.where(
#         map_condition, np.maximum(df['MAP'],df[ins2]),
#         np.where(
#             alp0_condition, np.maximum(df[ins2],df[out1]),
#             np.maximum(
#                 np.minimum(df[ins1], df[ins2]),
#                 df[out1])))
# # fun1.3
# def return_max_out(df,str1, ins1, ins2, out1):
#     df[str1] = np.where(
#         map_condition, np.maximum(df['MAP'],df[out1]),
#         np.where(
#             alp0_condition, np.maximum(df[ins2],df[out1]),
#             np.maximum(
#                 np.minimum(df[ins1], df[ins2]),
#                 df[out1])))
# # fun 2
# def cashback(df):
#     df['Cashback'] = (df['WFS Fee'] / 0.85).apply(lambda x: math.ceil(x))
#     df.loc[df['Cashback'] < 10, 'Cashback']= df['Cashback']+1
#     df.loc[(df['Cashback']<10) | (df['Cashback']>10), 'Cashback'] = df['Cashback']-0.01

# # fun 3
# def return_mapmax(df,str1,arg1):
#     df[str1] = np.where(
#         map_condition,
#         np.maximum(df['MAP'], df[arg1]),
#         df[arg1])

In [54]:
# # ----- Season Base -----

# # --- margin ---
# margin_percentages = [25,30]

# for margin_percentage in margin_percentages:
#     margin_column_name = f'{margin_percentage}% Margin'
#     df_season[margin_column_name] = round(df_season['Cost'] * (1 / (0.85 - margin_percentage / 100)), 2)
#     df_season[margin_column_name] = np.where(df_season[margin_column_name] < 10, df_season[margin_column_name] + 1.2, df_season[margin_column_name])

# # ---------------

# # cashback
# cashback(df_season)

# # lowest
# # change- use func
# df_season['Lowest Price'] = np.where((df_season['MAP Enforced'] == True) & (df_season['MAP'] > 0),
#                                      np.maximum(df_season['Storage+Cost'], df_season['MAP']),
#                                      df_season['Storage+Cost'])


# map_condition = (df_season['MAP Enforced'] == True) & (df_season['MAP'] > 0)
# alp0_condition = (df_season['Lowest Amazon Price'] == 0)
# # highest
# return_mapmax(df_season,'Highest Price','30% Margin')
# # current
# return_mapmax(df_season,'Current','25% Margin')

In [55]:
# # ----- Normal Season (df_45/ 0 sales)-----

# # --- margin ---
# margin_percentages = [10,25,30]

# for margin_percentage in margin_percentages:
#     margin_column_name = f'{margin_percentage}% Margin'
#     df_45[margin_column_name] = round(df_45['Cost'] * (1 / (0.85 - margin_percentage / 100)), 2)
#     df_45[margin_column_name] = np.where(df_45[margin_column_name] < 10, df_45[margin_column_name] + 1.2, df_45[margin_column_name])

# # ---------------

# # cashback
# #cashback(df_45)

# map_condition = (df_45['MAP Enforced'] == True) & (df_45['MAP'] > 0)
# alp0_condition = (df_45['Lowest Amazon Price'] == 0)

# # lowest
# return_mapmax(df_45,'Lowest Price','10% Margin')
# # highest
# return_mapmax(df_45,'Highest Price','30% Margin')
# # current
# return_mapmax(df_45,'Current','25% Margin')

In [56]:
# # ----- Normal Season (df) -----
# # --- margin ---
# margin_percentages = [10,25,30]

# for margin_percentage in margin_percentages:
#     margin_column_name = f'{margin_percentage}% Margin'
#     df[margin_column_name] = round(df['Cost'] * (1 / (0.85 - margin_percentage / 100)), 2)
#     df[margin_column_name] = np.where(df[margin_column_name] < 10, df[margin_column_name] + 1.2, df[margin_column_name])

# # ---------------

# # cashback
# #cashback(df)

# # lowest, highest, current
# map_condition = (df['MAP Enforced'] == True) & (df['MAP'] > 0)
# alp0_condition = (df['Lowest Amazon Price'] == 0)

# # lowest
# return_mapmax(df,'Lowest Price','10% Margin')
# # highest
# return_mapmax(df,'Highest Price','30% Margin')
# # current
# return_mapmax(df,'Current','25% Margin')

In [57]:
# condition = df['Lowest Price'] == df['Highest Price']
# df.loc[condition, 'Highest Price'] = df.loc[condition, 'Highest Price'] + 0.01

# condition = df_45['Lowest Price'] == df_45['Highest Price']
# df_45.loc[condition, 'Highest Price'] = df_45.loc[condition, 'Highest Price'] + 0.01

# condition = df_season['Lowest Price'] == df_season['Highest Price']
# df_season.loc[condition, 'Highest Price'] = df_season.loc[condition, 'Highest Price'] + 0.01

In [58]:
# # write other sheets to main
# try:
#     # ----- Seasonal + Seasonal BF -----
#     with pd.ExcelWriter('Re-price Table.xlsx', engine='openpyxl', mode='a') as writer:

#         # export sheet 2.--- Non Season
#         df.to_excel(writer, index=False, sheet_name='Non-Season')
#         # export sheet 3.--- Non Season
#         df_45.to_excel(writer, index=False, sheet_name='Non-Season(0 Sales)')
#         # export sheet 4.--- Normal Season
#         df_season.to_excel(writer, index=False, sheet_name='Season')
#         # export sheet 5.--- Incentive
#         df_incentive.to_excel(writer, index=False, sheet_name='Incentive')
#         # export sheet 6.--- Fix
#         df_fix.to_excel(writer, index=False, sheet_name='Fix Price')
#         # export sheet 7.--- NaN
#         df_nan.to_excel(writer, index=False, sheet_name='NaN')

# except PermissionError:
#     print("Please close the Excel file before running the script.")

# except Exception as e:
#     print(f"An error occurred: {e}")

In [59]:
# df_season[df_season['Lowest Price']>df_season['Highest Price']][['SKU','Lowest Amazon Price','Cost','25% Margin','Lowest Price','Highest Price']]

In [60]:
# df_season.columns